## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [58]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [59]:
# answer goes here
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv')
df



,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Álex Abrines,SG,25,OKC,31,2,19.0,1.8,5.1,0.357,1.3,4.1,0.323,0.5,1.0,0.500,0.487,0.4,0.4,0.923,0.2,1.4,1.5,0.6,0.5,0.2,0.5,1.7,5.3
1,2019,Quincy Acy,PF,28,PHO,10,0,12.3,0.4,1.8,0.222,0.2,1.5,0.133,0.2,0.3,0.667,0.278,0.7,1.0,0.700,0.3,2.2,2.5,0.8,0.1,0.4,0.4,2.4,1.7
2,2019,Jaylen Adams,PG,22,ATL,34,1,12.6,1.1,3.2,0.345,0.7,2.2,0.338,0.4,1.1,0.361,0.459,0.2,0.3,0.778,0.3,1.4,1.8,1.9,0.4,0.1,0.8,1.3,3.2
3,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
4,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,2016,Joe Young,PG,23,IND,41,0,9.4,1.5,4.1,0.367,0.2,1.1,0.217,1.3,3.0,0.423,0.396,0.5,0.6,0.800,0.1,1.1,1.2,1.6,0.4,0.0,0.8,0.7,3.8
2137,2016,Nick Young,SG,30,LAL,54,2,19.1,2.3,6.9,0.339,1.4,4.4,0.325,0.9,2.5,0.363,0.442,1.2,1.4,0.829,0.3,1.5,1.8,0.6,0.4,0.1,0.6,0.9,7.3
2138,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1
2139,2016,Cody Zeller,C,23,CHO,73,60,24.3,3.2,6.0,0.529,0.0,0.1,0.100,3.2,5.8,0.539,0.530,2.4,3.2,0.754,1.9,4.3,6.2,1.0,0.8,0.9,0.9,2.8,8.7


The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [60]:
# answer goes here
df = df[df['GS'] >= 20]
df = df[df['MP'] >= 10]
df = df.reset_index(drop=True)
df



,Seas,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2019,Steven Adams,C,25,OKC,80,80,33.4,6.0,10.1,0.595,0.0,0.0,0.000,6.0,10.1,0.596,0.595,1.8,3.7,0.500,4.9,4.6,9.5,1.6,1.5,1.0,1.7,2.6,13.9
1,2019,Bam Adebayo,C,21,MIA,82,28,23.3,3.4,5.9,0.576,0.0,0.2,0.200,3.4,5.7,0.588,0.579,2.0,2.8,0.735,2.0,5.3,7.3,2.2,0.9,0.8,1.5,2.5,8.9
2,2019,LaMarcus Aldridge,C,33,SAS,81,81,33.2,8.4,16.3,0.519,0.1,0.5,0.238,8.3,15.8,0.528,0.522,4.3,5.1,0.847,3.1,6.1,9.2,2.4,0.5,1.3,1.8,2.2,21.3
3,2019,Jarrett Allen,C,20,BRK,80,80,26.2,4.2,7.1,0.590,0.1,0.6,0.133,4.1,6.5,0.629,0.595,2.5,3.5,0.709,2.4,6.0,8.4,1.4,0.5,1.5,1.3,2.3,10.9
4,2019,Al-Farouq Aminu,PF,28,POR,81,81,28.3,3.2,7.3,0.433,1.2,3.5,0.343,2.0,3.9,0.514,0.514,1.9,2.1,0.867,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,2016,Deron Williams,PG,31,DAL,65,63,32.4,4.9,11.9,0.414,1.5,4.3,0.344,3.4,7.6,0.454,0.477,2.8,3.2,0.869,0.3,2.6,2.9,5.8,0.9,0.2,2.3,2.4,14.1
800,2016,Lou Williams,SG,29,LAL,67,35,28.5,4.2,10.3,0.408,1.6,4.8,0.344,2.6,5.6,0.464,0.488,5.3,6.3,0.830,0.5,2.1,2.5,2.5,0.9,0.3,1.6,1.6,15.3
801,2016,Marvin Williams,PF,29,CHO,81,81,28.9,4.2,9.2,0.452,1.9,4.7,0.402,2.3,4.6,0.504,0.554,1.5,1.8,0.833,1.6,4.9,6.4,1.4,0.7,1.0,0.8,1.6,11.7
802,2016,Thaddeus Young,PF,27,BRK,73,73,33.0,6.8,13.2,0.514,0.1,0.4,0.233,6.7,12.8,0.523,0.518,1.4,2.2,0.644,2.4,6.6,9.0,1.8,1.5,0.5,1.9,2.5,15.1


Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [61]:
# answer goes here
X = df[['PTS','TRB','AST','STL','BLK','TOV']]
X




,PTS,TRB,AST,STL,BLK,TOV
0,13.9,9.5,1.6,1.5,1.0,1.7
1,8.9,7.3,2.2,0.9,0.8,1.5
2,21.3,9.2,2.4,0.5,1.3,1.8
3,10.9,8.4,1.4,0.5,1.5,1.3
4,9.4,7.5,1.3,0.8,0.4,0.9
...,...,...,...,...,...,...
799,14.1,2.9,5.8,0.9,0.2,2.3
800,15.3,2.5,2.5,0.9,0.3,1.6
801,11.7,6.4,1.4,0.7,1.0,0.8
802,15.1,9.0,1.8,1.5,0.5,1.9


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [62]:
# answer goes here
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
X_scale

,PTS,TRB,AST,STL,BLK,TOV
0,0.114638,1.612909,-0.617816,1.393220,0.824001,0.023511
1,-0.744985,0.768627,-0.330886,-0.050271,0.420162,-0.218831
2,1.386880,1.497780,-0.235242,-1.012598,1.429758,0.144681
3,-0.401136,1.190768,-0.713459,-1.012598,1.833596,-0.461172
4,-0.659023,0.845380,-0.761281,-0.290853,-0.387514,-0.945854
...,...,...,...,...,...,...
799,0.149023,-0.919937,1.390695,-0.050271,-0.791352,0.750535
800,0.355332,-1.073443,-0.187421,-0.050271,-0.589433,-0.097660
801,-0.263596,0.423239,-0.713459,-0.531435,0.824001,-1.067025
802,0.320947,1.421027,-0.522172,1.393220,-0.185595,0.265852


Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [63]:
kmeans = KMeans(n_clusters=3)
X_scale['cluster'] = kmeans.fit_predict(X_scale)
centers = pd.DataFrame(kmeans.cluster_centers_,columns=X.columns)
center_inverse = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
center_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,10.505405,3.949775,2.063739,0.804730,0.374324,1.235135
1,13.735359,8.605525,1.914917,0.777348,1.230387,1.668508
2,19.491620,5.293855,5.934078,1.354190,0.486034,2.797765


In [64]:
X = df[['PTS','TRB','AST','STL','BLK','TOV']]
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
kmeans = KMeans(n_clusters=7)
X_scale['cluster'] = kmeans.fit_predict(X_scale)
centers = pd.DataFrame(kmeans.cluster_centers_,columns=X.columns)
center_inverse = pd.DataFrame(scale.inverse_transform(centers), columns=X.columns)
center_inverse.style.background_gradient()

,PTS,TRB,AST,STL,BLK,TOV
0,13.827374,3.707821,3.059218,0.840782,0.278212,1.660894
1,7.510989,3.556593,1.318681,0.604396,0.374725,0.865934
2,23.559459,8.008108,7.524324,1.648649,0.821622,3.791892
3,11.663014,7.378082,1.726027,0.656164,0.943836,1.421918
4,18.533871,4.590323,5.735484,1.291935,0.373387,2.591935
5,11.465385,5.070513,2.089744,1.400000,0.521795,1.317949
6,17.765517,10.512069,2.291379,0.925862,1.770690,2.141379


In [65]:
X = df[['PTS','TRB','AST','STL','BLK','TOV']]
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
gmm = GaussianMixture(n_components=3)
gmm.fit(X_scale)
X['gmm_cluster'] = gmm.predict(X_scale)
probs = pd.DataFrame(gmm.predict_proba(X_scale))
X = pd.concat([X, probs], axis =1)
gmm_center = pd.DataFrame(gmm.means_, columns=X_scale.columns)
gmm_center_inverse = pd.DataFrame(
    scale.inverse_transform(gmm_center), columns=X_scale.columns
)
gmm_center_inverse.style.background_gradient()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,PTS,TRB,AST,STL,BLK,TOV
0,10.239585,4.476528,1.582674,0.780572,0.428468,1.105656
1,15.898592,4.178026,4.812924,1.176953,0.371760,2.246579
2,14.449177,8.481379,2.259574,0.775804,1.221910,1.830776


In [66]:
X = df[['PTS','TRB','AST','STL','BLK','TOV']]
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
gmm = GaussianMixture(n_components=7)
gmm.fit(X_scale)
X['gmm_cluster'] = gmm.predict(X_scale)
probs = pd.DataFrame(gmm.predict_proba(X_scale))
X = pd.concat([X, probs], axis =1)
gmm_center = pd.DataFrame(gmm.means_, columns=X_scale.columns)
gmm_center_inverse = pd.DataFrame(
    scale.inverse_transform(gmm_center), columns=X_scale.columns
)
gmm_center_inverse.style.background_gradient()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,PTS,TRB,AST,STL,BLK,TOV
0,7.482534,3.648663,1.221779,0.639675,0.416430,0.840942
1,17.305517,4.587433,5.576382,1.432368,0.384483,2.552388
2,16.696525,9.277908,3.105392,0.837220,1.356119,2.151195
3,11.383194,7.160211,1.352531,0.609114,1.012691,1.327771
4,13.988674,3.006335,3.889885,0.860603,0.200049,1.762179
5,20.704936,8.797532,6.841012,1.608020,0.972523,3.502430
6,13.146758,4.921594,2.139140,1.004740,0.435388,1.457501


Predict the likelihood of each player belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those players.

In [67]:
# answer goes here
X = df[['PTS','TRB','AST','STL','BLK','TOV']]
X_scale = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
gmm = GaussianMixture(n_components=3)
gmm.fit(X_scale)
X['gmm_cluster'] = gmm.predict(X_scale)
probs = pd.DataFrame(gmm.predict_proba(X_scale))
X = pd.concat([X, probs], axis =1)
entropy = entr(probs)
entropy['entropy'] = entropy[0] + entropy[1] + entropy[2]
entropy['entropy']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0      0.532723
1      0.675997
2      0.004761
3      0.022368
4      0.354979
         ...   
799    0.008336
800    0.676313
801    0.608277
802    0.494321
803    0.144825
Name: entropy, Length: 804, dtype: float64

In [68]:
X = pd.concat([X,entropy['entropy']],axis=1)
X

,PTS,TRB,AST,STL,BLK,TOV,gmm_cluster,0,1,2,entropy
0,13.9,9.5,1.6,1.5,1.0,1.7,0,7.781604e-01,4.917650e-04,0.221348,0.532723
1,8.9,7.3,2.2,0.9,0.8,1.5,0,6.224176e-01,2.125529e-03,0.375457,0.675997
2,21.3,9.2,2.4,0.5,1.3,1.8,2,5.609724e-04,4.481060e-08,0.999439,0.004761
3,10.9,8.4,1.4,0.5,1.5,1.3,2,3.338223e-03,1.133751e-09,0.996662,0.022368
4,9.4,7.5,1.3,0.8,0.4,0.9,0,8.863883e-01,1.252464e-04,0.113486,0.354979
...,...,...,...,...,...,...,...,...,...,...,...
799,14.1,2.9,5.8,0.9,0.2,2.3,1,5.825387e-15,9.989376e-01,0.001062,0.008336
800,15.3,2.5,2.5,0.9,0.3,1.6,0,6.184747e-01,3.797123e-01,0.001813,0.676313
801,11.7,6.4,1.4,0.7,1.0,0.8,0,7.053126e-01,2.448026e-04,0.294443,0.608277
802,15.1,9.0,1.8,1.5,0.5,1.9,0,8.136153e-01,2.528205e-03,0.183856,0.494321


In [69]:
X = X.sort_values(by='entropy', ascending=True)
X.head(5)
#full confidence top 5

,PTS,TRB,AST,STL,BLK,TOV,gmm_cluster,0,1,2,entropy
797,14.2,11.8,0.4,0.6,3.7,1.9,2,1.666177e-38,9.539871e-49,1.0,1.449369e-36
182,13.3,7.2,1.6,0.8,2.7,1.4,2,4.972994e-22,2.364390e-25,1.0,2.440736e-20
252,28.1,11.1,2.3,1.5,2.6,2.2,2,2.161346e-21,4.509266e-19,1.0,1.915133e-17
351,22.7,6.6,1.2,0.8,2.4,1.9,2,2.283088e-23,4.547406e-19,1.0,1.920696e-17
454,20.2,7.8,2.1,0.9,2.5,3.8,2,2.988505e-31,1.370601e-17,1.0,5.321864e-16


In [71]:
X = X.sort_values(by='entropy', ascending=False)
X.head(5)
#full confidence top 5

,PTS,TRB,AST,STL,BLK,TOV,gmm_cluster,0,1,2,entropy
13,11.5,4.6,2.9,0.4,0.5,1.5,2,0.298850,0.262178,0.438972,1.073355
741,12.0,5.5,1.9,0.9,0.5,2.1,0,0.450060,0.369503,0.180436,1.036167
343,11.5,5.7,2.7,0.8,0.5,1.9,0,0.578802,0.175288,0.245911,0.966680
109,7.8,3.2,2.5,0.6,0.5,1.3,0,0.562921,0.353766,0.083313,0.898114
406,22.4,5.9,2.9,0.8,0.5,2.1,0,0.475990,0.454327,0.069683,0.897413
